In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import csv
import time
from tqdm import tqdm

In [2]:
# Define the API URL
url = r"https://www.nseindia.com/api/historical/cm/equity?symbol=20MICRONS&series=[%22EQ%22]&from=01-01-2015&to=01-01-2016"

# Define the headers (NSE website requires specific headers)
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Accept-Language': 'en-US,en;q=0.9',
    'Referer': 'https://www.nseindia.com/',
    'X-Requested-With': 'XMLHttpRequest',
    'Connection': 'keep-alive'
}

In [3]:
# Create a session to persist certain parameters across requests
session = requests.Session()

# Send an initial request to establish the session
session.get("https://www.nseindia.com", headers=headers)

# Send the GET request to the API
response = session.get(url, headers=headers)

In [4]:
# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON data
    data = response.json()
    
    # If the data is structured as a list of records, convert it to a DataFrame
    if 'data' in data:
        df = pd.DataFrame(data['data'])
        print(df.head())
    else:
        print("The JSON structure is not as expected.")
else:
    print(f"Failed to retrieve data: {response.status_code}")

                        _id  CH_SYMBOL CH_SERIES CH_MARKET_TYPE  \
0  641eb9e05510ab0007edc609  20MICRONS        EQ              N   
1  641eb9d6c151a800069a8944  20MICRONS        EQ              N   
2  641eb9cc3dfde30007ccc115  20MICRONS        EQ              N   
3  641eb9c24438f60007b58923  20MICRONS        EQ              N   
4  641eb9b74708d500072556bb  20MICRONS        EQ              N   

   CH_TRADE_HIGH_PRICE  CH_TRADE_LOW_PRICE  CH_OPENING_PRICE  \
0                36.30               34.60             36.00   
1                38.75               36.25             36.65   
2                38.45               32.25             32.25   
3                34.40               32.15             33.50   
4                34.90               31.80             31.90   

   CH_CLOSING_PRICE  CH_LAST_TRADED_PRICE  CH_PREVIOUS_CLS_PRICE  ...  \
0             35.20                 35.00                  36.50  ...   
1             36.50                 36.40                  36.55  

In [6]:
symbols = pd.read_csv("NSE_securities.csv")
symbols = symbols["SYMBOL"].iloc[331:661]

In [7]:
symbols.info()

<class 'pandas.core.series.Series'>
RangeIndex: 330 entries, 331 to 660
Series name: SYMBOL
Non-Null Count  Dtype 
--------------  ----- 
330 non-null    object
dtypes: object(1)
memory usage: 2.7+ KB


In [8]:
# Define the base URL and headers
base_url = r"https://www.nseindia.com/api/historical/cm/equity?symbol={symbol}&series=[%22EQ%22]&from={start_date}&to={end_date}"

# Define the headers
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Accept-Language': 'en-US,en;q=0.9',
    'Referer': 'https://www.nseindia.com/',
    'X-Requested-With': 'XMLHttpRequest',
    'Connection': 'keep-alive'
}

session = requests.Session()
session.get("https://www.nseindia.com", headers=headers)

all_stock_data = pd.DataFrame()
forbidden_company = []

for symbol in tqdm(symbols):
    stock_data = pd.DataFrame()
    # print(symbol)
    for start_year in range(2015, 2024):
        time.sleep(1)
        start_date = f"01-01-{start_year}"
        end_date = f"31-12-{start_year}"
        url = base_url.format(symbol=symbol, start_date=start_date, end_date=end_date)

        try:
            response = session.get(url, headers=headers, timeout=2)
            # Check if the request was successful
            if response.status_code == 200:
                data = response.json()
                if 'data' in data:
                    df = pd.DataFrame(data['data'])
                    df["Symbol"] = symbol

                    required_columns = ['Symbol', 'CH_TIMESTAMP', 'CH_OPENING_PRICE', 'CH_TRADE_HIGH_PRICE',
                                        'CH_TRADE_LOW_PRICE', 'CH_CLOSING_PRICE', 'CH_TOT_TRADED_QTY', 'CH_TOTAL_TRADES']

                    if required_columns in list(df.columns):
                        df = df[required_columns]
                        stock_data = pd.concat([stock_data, df], ignore_index=True)
                    else:
                        if symbol not in forbidden_company:
                            forbidden_company.append(symbol)
                        else:
                            continue
                else:
                    print("The JSON structure is not as expected.")
            else:
                if symbol not in forbidden_company:
                    forbidden_company.append(symbol)
                else:
                    continue
        except requests.exceptions.Timeout:
            if symbol not in forbidden_company:
                forbidden_company.append(symbol)
            else:
                continue
        except Exception as e:
            if symbol not in forbidden_company:
                forbidden_company.append(symbol)
            else:
                continue

    all_stock_data = pd.concat([all_stock_data, stock_data], ignore_index=True)

  0%|          | 0/330 [00:08<?, ?it/s]


KeyboardInterrupt: 

In [10]:
for symbol in tqdm(symbols):
    stock_data = pd.DataFrame()
    # print(symbol)
    for start_year in range(2024, 2025):
        time.sleep(1)
        start_date = f"01-01-{start_year}"
        end_date = f"30-06-{start_year}"
        url = base_url.format(symbol=symbol, start_date=start_date, end_date=end_date)

        try:
            response = session.get(url, headers=headers, timeout=2)
            # Check if the request was successful
            if response.status_code == 200:
                data = response.json()
                if 'data' in data:
                    df = pd.DataFrame(data['data'])
                    df["Symbol"] = symbol

                    required_columns = ['Symbol', 'CH_TIMESTAMP', 'CH_OPENING_PRICE', 'CH_TRADE_HIGH_PRICE',
                                        'CH_TRADE_LOW_PRICE', 'CH_CLOSING_PRICE', 'CH_TOT_TRADED_QTY', 'CH_TOTAL_TRADES']

                    if required_columns in list(df.columns):
                        df = df[required_columns]
                        stock_data = pd.concat([stock_data, df], ignore_index=True)
                    else:
                        if symbol not in forbidden_company:
                            forbidden_company.append(symbol)
                        else:
                            continue
                else:
                    print("The JSON structure is not as expected.")
            else:
                if symbol not in forbidden_company:
                    forbidden_company.append(symbol)
                else:
                    continue
        except requests.exceptions.Timeout:
            if symbol not in forbidden_company:
                forbidden_company.append(symbol)
            else:
                continue
        except Exception as e:
            if symbol not in forbidden_company:
                forbidden_company.append(symbol)
            else:
                continue

    all_stock_data = pd.concat([all_stock_data, stock_data], ignore_index=True)

162

In [88]:
# Save the concatenated data to a CSV file
output_filename = 'NSE_all_stock_data.csv'
all_stock_data.to_csv(output_filename, index=False)

print(f"All data saved to {output_filename}")

<Response [200]>

In [11]:
forbidden_company, len(forbidden_company)

['CENTURYPLY',
 'CENTURYTEX',
 'CERA',
 'CEREBRAINT',
 'CESC',
 'CGCL',
 'CGPOWER',
 'CHALET',
 'CHAMBLFERT',
 'CHEMBOND',
 'CHEMCON',
 'CHEMFAB',
 'CHEMPLASTS',
 'CHENNPETRO',
 'CHEVIOT',
 'CHOICEIN',
 'CHOLAFIN',
 'CHOLAHLDNG',
 'CIEINDIA',
 'CIGNITITEC',
 'CINELINE',
 'CINEVISTA',
 'CIPLA',
 'CLEAN',
 'CLEDUCATE',
 'CLSEL',
 'CMSINFO',
 'COALINDIA',
 'COASTCORP',
 'COCHINSHIP',
 'COFFEEDAY',
 'COFORGE',
 'COLPAL',
 'COMPUSOFT',
 'COMSYN',
 'CONCOR',
 'CONCORDBIO',
 'CONFIPET',
 'CONSOFINVT',
 'CONTROLPR',
 'CORALFINAC',
 'CORDSCABLE',
 'COROMANDEL',
 'COSMOFIRST',
 'COUNCODOS',
 'CRAFTSMAN',
 'CREATIVE',
 'CREATIVEYE',
 'CREDITACC',
 'CREST',
 'CRISIL',
 'CROMPTON',
 'CROWN',
 'CSBBANK',
 'CSLFINANCE',
 'CTE',
 'CUB',
 'CUBEXTUB',
 'CUMMINSIND',
 'CUPID',
 'CYBERMEDIA',
 'CYBERTECH',
 'CYIENT',
 'CYIENTDLM',
 'DABUR',
 'DALBHARAT',
 'DALMIASUG',
 'DAMODARIND',
 'DANGEE',
 'DATAMATICS',
 'DATAPATTNS',
 'DAVANGERE',
 'DBCORP',
 'DBL',
 'DBOL',
 'DBREALTY',
 'DBSTOCKBRO',
 'DCAL',
 'DC